# 1. Download data from one specific asset
I am following the steps from this website: https://www.datasciencecentral.com/getting-historical-data-from-yahoo-finance-in-r/

In [1]:
stock <- "AAPL"

t1 <- as.integer(ISOdate(2010,1,17,hour=0))
t2 <- as.integer(ISOdate(2022,2,17,hour=0))

url <- paste("https://query1.finance.yahoo.com/v7/finance/download/",
 stock,
 "?period1=",
 as.integer(t1),
 "&period2=",
 as.integer(t2),
 "&interval=1d&events=history&includeAdjustedClose=true",
 sep="")
 url

[1] "https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=1263686400&period2=1645056000&interval=1d&events=history&includeAdjustedClose=true"

In [2]:
# Downloading the .csv file
data <- read.csv(url)
data['Date'] <- as.Date(data$Date)
head(data)

Date,Open,High,Low,Close,Adj.Close,Volume
2010-01-19,7.440357,7.685357,7.401429,7.680000,6.576185,730007600
2010-01-20,7.675357,7.698214,7.482143,7.561786,6.474962,612152800
2010-01-21,7.574286,7.618214,7.400357,7.431071,6.363034,608154400
2010-01-22,7.385000,7.410714,7.041429,7.062500,6.047437,881767600
2010-01-25,7.232500,7.310714,7.149643,7.252500,6.210130,1065699600
2010-01-26,7.355357,7.632500,7.235000,7.355000,6.297898,1867110000


In [3]:
# Take a look to data types
sapply(data, class)

Date      Open      High       Low     Close Adj.Close    Volume 
   "Date" "numeric" "numeric" "numeric" "numeric" "numeric" "integer"

In [4]:
# Keep only Adj.Close and change its name to AAPL:
AAPL <- data.frame(date=data$Date, AAPL=data$Adj.Close)
head(AAPL)

date,AAPL
2010-01-19,6.576185
2010-01-20,6.474962
2010-01-21,6.363034
2010-01-22,6.047437
2010-01-25,6.210130
2010-01-26,6.297898


# 2. Create a dataset of several stocks

In [5]:
# Choose the assets that will be included in the dataset
stocks <- c("AAPL", "AMZN", "MSFT", "GOOGL", "BRK-A", "JPM")
datalist <- list() 
t1 <- as.integer(ISOdate(2010,1,17,hour=0))
t2 <- as.integer(ISOdate(2022,2,17,hour=0))

for (stock in stocks) {
    url <- paste("https://query1.finance.yahoo.com/v7/finance/download/",
                 stock,
                 "?period1=",
                 as.integer(t1),
                 "&period2=",
                 as.integer(t2),
                 "&interval=1d&events=history&includeAdjustedClose=true",
                 sep="")
    data <- read.csv(url)
    data['Date'] <- as.Date(data$Date)    
    datalist[[stock]] <- data.frame(date=data$Date,price=data$Adj.Close)
}

In [6]:
# Merge
prices <- data.frame(datalist$AAPL)
colnames(prices) <- c('date','AAPL')
for (i in 2:length(datalist)) {
    temp <- datalist[[i]]
    colnames(temp) <- c('date', stocks[i])
    prices <- merge(prices,temp,by="date")
}
prices <- data.frame(prices)

In [7]:
tail(prices)

,date,AAPL,AMZN,MSFT,GOOGL,BRK.A,JPM
3038,2022-02-09,176.28,3223.79,310.5678,2831.84,483900,156.60
3039,2022-02-10,172.12,3180.07,301.7561,2772.40,480105,155.95
3040,2022-02-11,168.64,3065.87,294.4312,2685.65,479370,153.92
3041,2022-02-14,168.88,3103.34,294.3913,2710.52,472410,152.49
3042,2022-02-15,172.79,3130.21,299.8500,2732.17,473980,154.72
3043,2022-02-16,172.55,3162.01,299.5000,2754.76,474444,155.00


In [8]:
summary(prices[2:7])

      AAPL              AMZN             MSFT            GOOGL       
 Min.   :  5.873   Min.   : 108.6   Min.   : 17.90   Min.   : 218.3  
 1st Qu.: 16.347   1st Qu.: 259.3   1st Qu.: 25.37   1st Qu.: 379.6  
 Median : 26.273   Median : 616.9   Median : 47.26   Median : 731.6  
 Mean   : 42.416   Mean   :1091.1   Mean   : 84.16   Mean   : 895.8  
 3rd Qu.: 48.496   3rd Qu.:1765.4   3rd Qu.:108.43   3rd Qu.:1159.5  
 Max.   :181.778   Max.   :3731.4   Max.   :342.40   Max.   :2996.8  
     BRK.A             JPM        
 Min.   :100000   Min.   : 21.41  
 1st Qu.:145702   1st Qu.: 36.67  
 Median :216266   Median : 54.93  
 Mean   :231926   Mean   : 70.05  
 3rd Qu.:303000   3rd Qu.: 97.39  
 Max.   :485379   Max.   :170.76  